# Previsões com as variáveis independentes para verificar se o dataset consegue prever a variável dependente.

In [8]:
import os
import numpy as np
import pandas as pd

def load_txt_files_from_folders(root_directory):
    """
    Percorre todas as pastas dentro do diretório raiz e carrega arquivos .txt de cada pasta.

    Args:
        root_directory (str): Caminho para o diretório raiz.

    Returns:
        dict: Um dicionário com as informações carregadas.
              Chave = 'nome_da_pasta/nome_do_arquivo'
              Valor = (cabeçalho, np.array com os dados)
    """
    relevant_features_data = {}

    # Percorre todas as subpastas dentro do diretório raiz
    for folder_name in os.listdir(root_directory):
        folder_path = os.path.join(root_directory, folder_name)

        # Verifica se é uma pasta
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith('.txt'):
                    file_path = os.path.join(folder_path, filename)
                    try:
                        # Lendo os dados do arquivo
                        with open(file_path, 'r') as file:
                            lines = file.readlines()
                            header = lines[0].strip()  # Mantém o cabeçalho
                            data = [line.strip().split(',') for line in lines[1:] if line.strip()]  # Supondo CSV-like

                            # Adiciona ao dicionário com chave como 'pasta/arquivo'
                            key = f"{folder_name}/{filename}"
                            relevant_features_data[key] = (header, np.array(data, dtype=float))

                    except Exception as e:
                        print(f"Erro ao processar o arquivo {file_path}: {e}")
    
    return relevant_features_data


# Exemplo de uso:
root_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output"
datasets = load_txt_files_from_folders(root_directory)

# Exibindo o resumo dos arquivos carregados
for key, (header, data) in datasets.items():
    print(f"Arquivo processado: {key}")
    print(f"Header: {header}")
    print(f"Shape dos dados: {data.shape}")
    print("-" * 50)


Arquivo processado: 0-saida/cocomo81.txt
Header: rely,data,cplx,time,stor,virt,turn,acap,aexp,pcap,vexp,lexp,modp,tool,sced,loc,actual
Shape dos dados: (63, 17)
--------------------------------------------------
Arquivo processado: 0-saida/encoded_china.txt
Header: ID,AFP,Input,Output,Enquiry,File,Interface,Added,Changed,Deleted,PDR_AFP,PDR_UFP,NPDR_AFP,NPDU_UFP,Duration,N_effort,Resource_1,Resource_2,Resource_3,Resource_4,Dev.Type_0,Effort
Shape dos dados: (499, 22)
--------------------------------------------------
Arquivo processado: 0-saida/encoded_desharnais.txt
Header: YearEnd,Transactions,Entities,PointsAdjust,Envergure,TeamExp_0,TeamExp_1,TeamExp_2,TeamExp_3,TeamExp_4,TeamExp_?,ManagerExp_0,ManagerExp_1,ManagerExp_2,ManagerExp_3,ManagerExp_4,ManagerExp_5,ManagerExp_7,ManagerExp_?,Language_1,Language_2,Language_3,Effort
Shape dos dados: (81, 23)
--------------------------------------------------
Arquivo processado: 0-saida/encoded_maxwell.txt
Header: size_D,duration_D,nlan,t01,t

In [3]:
!pip install matplotlib


     ---------------------------------------- 0.0/168.5 kB ? eta -:--:--
     --------- ----------------------------- 41.0/168.5 kB 2.0 MB/s eta 0:00:01
     ------------------------------------ - 163.8/168.5 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 168.5/168.5 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.0 MB 20.3 MB/s eta 0:00:01
   -------------- ------------------------- 3.0/8.0 MB 32.3 MB/s eta 0:00:01
   ------------------------- -------------- 5.0/8.0 MB 35.8 MB/s eta 0:00:01
   ------------------------------------ --- 7.2/8.0 MB 38.7 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 36.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/219.8 kB ? eta -:--:--
   --------------------------------------- 219.8/219.8 kB 14.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ----------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\CALEO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Função para realizar a previsão e comparar com os resultados esperados
def predict_and_evaluate(data, header):
    # Converter o array para um DataFrame
    df = pd.DataFrame(data, columns=header.split(','))

    # Separar variáveis independentes (X) e variável dependente (y)
    X = df.iloc[:, :-1].values  # Todas as colunas, exceto a última
    y = df.iloc[:, -1].values   # Última coluna (variável dependente)

    # Dividir em dados de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Treinar o modelo de Regressão Linear
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Fazer previsões
    y_pred = model.predict(X_test)

    # Avaliar o modelo
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Raiz do Erro Quadrático Médio
    r2 = r2_score(y_test, y_pred)

    return rmse, r2

# Aplicar o modelo para cada dataset
results = []
for key, (header, data) in datasets.items():
    print(f"\nProcessando arquivo: {key}")
    best_rmse, best_r2 = float('inf'), float('-inf')  # Inicializa os melhores valores

    for _ in range(30):  # Executa 30 vezes
        try:
            rmse, r2 = predict_and_evaluate(data, header)

            # Atualiza os melhores valores
            if r2 > best_r2 or (r2 == best_r2 and rmse < best_rmse):
                best_rmse, best_r2 = rmse, r2

        except Exception as e:
            print(f"Erro ao processar {key} na iteração {_ + 1}: {e}")

    # Armazena o melhor resultado para a base
    results.append((key, best_rmse, best_r2))

# Salvar todas as saídas, ranqueadas das melhores para as piores
results_sorted = sorted(results, key=lambda x: x[2], reverse=True)

# Converter os resultados para um DataFrame
all_results_df = pd.DataFrame(results_sorted, columns=["Arquivo", "RMSE", "R²"])

# Salvar o DataFrame em um arquivo Excel
all_results_df.to_excel("resultados_melhores_rankeados.xlsx", index=False)

print("Todos os resultados foram salvos em 'resultados_melhores_rankeados.xlsx'")



Processando arquivo: 0-saida/cocomo81.txt

Processando arquivo: 0-saida/encoded_china.txt

Processando arquivo: 0-saida/encoded_desharnais.txt

Processando arquivo: 0-saida/encoded_maxwell.txt

Processando arquivo: 1-saida/2-saida_cocomo81.txt

Processando arquivo: 1-saida/2-saida_encoded_china.txt

Processando arquivo: 1-saida/2-saida_encoded_desharnais.txt

Processando arquivo: 1-saida/2-saida_encoded_maxwell.txt

Processando arquivo: 10-saida/cocomo81.txt

Processando arquivo: 10-saida/encoded_china.txt

Processando arquivo: 10-saida/encoded_desharnais.txt

Processando arquivo: 10-saida/encoded_maxwell.txt

Processando arquivo: 11-saida/2-saida_cocomo81.txt

Processando arquivo: 11-saida/2-saida_encoded_china.txt

Processando arquivo: 11-saida/2-saida_encoded_desharnais.txt

Processando arquivo: 11-saida/2-saida_encoded_maxwell.txt

Processando arquivo: 12-saida/2-saida_cocomo81.txt

Processando arquivo: 12-saida/2-saida_encoded_china.txt

Processando arquivo: 12-saida/2-saida_enco